# Notebook to generate PCA Graphics

### Libraries import

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, interact

In [ ]:
def show_all_videos(real_videos, fake_videos, substraction_videos):
    assert len(real_videos) == len(fake_videos) == len(substraction_videos)
    for i in range(len(real_videos)):
        assert real_videos[i].shape == fake_videos[i].shape == substraction_videos[i].shape
    
    frame_slider = IntSlider(min=1, max=real_videos[0].shape[0], step=1)
    volume_slider = IntSlider(min=1, max=len(real_videos), step=1)

    def update_frame_max(*args):
        frame_slider.max = real_videos[volume_slider.value].shape[0]
    volume_slider.observe(update_frame_max, 'value')
    
    interact(lambda volume, frame: plt.imshow(real_videos[volume-1][frame-1]),
        volume=volume_slider, frame=frame_slider
    )
    interact(lambda volume, frame: plt.imshow(fake_videos[volume-1][frame-1]),
        volume=volume_slider, frame=frame_slider
    )
    interact(lambda volume, frame: plt.imshow(substraction_videos[volume-1][frame-1]),
        volume=volume_slider, frame=frame_slider
    )

### Experiment selection

In [ ]:
experiment_id = "0006"
root_path = "../../results/Ganomaly_3D/"
for i in sorted(os.listdir(root_path)):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

### Reconstructions Loading

In [ ]:
base_path = os.path.join(experiment_folder, "outputs/samples/")
for t in ["real", "fake", "substraction"]:
    for c in ["normal", "abnormal"]:
        globals()["{}_{}_videos".format(t, c)] = []

for t in ["real", "fake", "substraction"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if len(os.listdir(os.path.join(base_path, t, m, "normal"))) != 0:
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]
        for c in classes:
            videos_path = os.path.join(base_path, t, m, c)
            for video_folder in sorted(os.listdir(videos_path)):
                video = []
                video_path = os.path.join(videos_path, video_folder)
                for frame in sorted(os.listdir(video_path)):
                    video.append(cv2.cvtColor(
                        cv2.imread(
                            os.path.join(video_path, frame)
                        ), cv2.COLOR_BGR2RGB
                    ))
                globals()["{}_{}_videos".format(t, c)].append(np.r_[video])
len(real_normal_videos), len(real_abnormal_videos)

### Visualization

In [ ]:
videos = "normal"
show_all_videos(
    globals()["real_{}_videos".format(videos)],
    globals()["fake_{}_videos".format(videos)],
    globals()["substraction_{}_videos".format(videos)],
)